In [4]:
import pandas as pd
import numpy as np
import geoip2.database

In [5]:
merged_df = pd.read_csv('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/feb5-12_merged.csv')
merged_df = merged_df.replace([np.inf, -np.inf], np.nan)
merged_df = merged_df.dropna()

In [6]:
reader = geoip2.database.Reader('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/GeoLite2-City.mmdb')
# Build location info
locations = []

for ip in merged_df['Src IP']:
    try:
        response = reader.city(ip)
        country = response.country.name or "Unknown Country"
        city = response.city.name or "Unknown City"

        location_str = f"{country}"
    except Exception as e:
        location_str = f"Error: {e}"

    locations.append(location_str)

# Add to DataFrame
merged_df['Location'] = locations

In [7]:
'''
90% of our data is represented in 10 columns, as such we are choosing to limit our model to those countries
'''
merged_df = merged_df[(merged_df['Location'] == 'Iran') | (merged_df['Location'] == 'United States') | (merged_df['Location'] == 'Hong Kong') | (merged_df['Location'] == 'China') | (merged_df['Location'] == 'Germany') | (merged_df['Location'] == 'Japan') | (merged_df['Location'] == 'Italy') | (merged_df['Location'] == 'France') | (merged_df['Location'] == 'Singapore') | (merged_df['Location'] == 'United Kingdom')]

In [8]:
threshold = 0.98
dropped = []
for col in merged_df.columns:
    top_freq = merged_df[col].value_counts(normalize=True, dropna=False).iloc[0]
    if top_freq > threshold:
        dropped.append(col)
        merged_df = merged_df.drop(columns=col)
merged_df

,Flow ID,Src IP,Src Port,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,...,Fwd Act Data Pkts,Active Mean,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,day,Location
0,129.226.189.21-10.1.0.4-36211-123-17,129.226.189.21,36211,2023-02-05 23:02:50,30009444,3056,4,24448.0,1472.0,8.0,...,3055,0.0,0.0,0.0,1.675638e+15,0.0,1.675638e+15,1.675638e+15,5,Hong Kong
1,129.226.189.21-10.1.0.4-40528-123-17,129.226.189.21,40528,2023-02-05 23:02:50,30018257,3045,4,24360.0,1472.0,8.0,...,3044,0.0,0.0,0.0,1.675638e+15,0.0,1.675638e+15,1.675638e+15,5,Hong Kong
2,129.226.189.21-10.1.0.4-33823-123-17,129.226.189.21,33823,2023-02-05 23:02:50,30010489,3163,4,25304.0,1472.0,8.0,...,3162,0.0,0.0,0.0,1.675638e+15,0.0,1.675638e+15,1.675638e+15,5,Hong Kong
3,129.226.189.21-10.1.0.4-34706-123-17,129.226.189.21,34706,2023-02-05 23:02:50,30013069,3025,4,24200.0,1472.0,8.0,...,3024,0.0,0.0,0.0,1.675638e+15,0.0,1.675638e+15,1.675638e+15,5,Hong Kong
4,129.226.189.21-10.1.0.4-33563-123-17,129.226.189.21,33563,2023-02-05 23:02:50,30016745,3039,4,24312.0,1472.0,8.0,...,3038,0.0,0.0,0.0,1.675638e+15,0.0,1.675638e+15,1.675638e+15,5,Hong Kong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739202,36.156.28.131-10.1.0.4-52748-161-6,36.156.28.131,52748,2023-02-12 23:20:01,66,1,1,0.0,0.0,0.0,...,0,0.0,0.0,0.0,1.676244e+15,0.0,1.676244e+15,1.676244e+15,12,China
1739203,172.254.82.45-10.1.0.4-33190-23-6,172.254.82.45,33190,2023-02-12 23:20:26,123,1,1,0.0,0.0,0.0,...,0,0.0,0.0,0.0,1.676244e+15,0.0,1.676244e+15,1.676244e+15,12,United States
1739205,45.136.12.26-10.1.0.4-19211-123-17,45.136.12.26,19211,2023-02-12 23:21:04,19783704,1192,0,9536.0,0.0,8.0,...,1191,0.0,0.0,0.0,1.676244e+15,0.0,1.676244e+15,1.676244e+15,12,Hong Kong
1739206,45.136.12.26-10.1.0.4-21363-123-17,45.136.12.26,21363,2023-02-12 23:21:04,19781062,1213,0,9704.0,0.0,8.0,...,1212,0.0,0.0,0.0,1.676244e+15,0.0,1.676244e+15,1.676244e+15,12,Hong Kong


In [9]:
merged_df['Country'] = merged_df['Location'].map({'Iran':1,'United States':2,'Hong Kong':3,'China':4,'Germany':5,'Japan':6,'Italy':7,'France':8,'Singapore':9,'United Kingdom':10})
merged_df = pd.get_dummies(merged_df,columns=['Location'], dtype='int')

In [10]:
merged_df = merged_df.drop(columns=['Flow ID','Src IP','Timestamp','day','Fwd Act Data Pkts', 'Fwd Header Length', 'Fwd Packet Length Mean','Total Length of Bwd Packet', 'Bwd Header Length', 'Bwd Segment Size Avg', 'Bwd Bytes/Bulk Avg','Bwd IAT Min','Bwd IAT Max', 'Bwd IAT Total','Packet Length Max','Packet Length Std','Packet Length Variance','Average Packet Size','Active Min','Active Max','Idle Min', 'Idle Std'])

In [11]:
merged_df.to_csv('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/Clean4Model')